In [1]:
using AbstractTrees
using D3Trees

using HTTP
using Sockets
using JSON

In [2]:
module MyNodes
depth(id::Int64) = floor(Int64, log2(id))
struct BTNode
    id::Int64
    expand_depth::Int64
end

function BTNode(id; Δd=2)
    d = depth(id)
    BTNode(id, d+Δd)
end
end

Main.MyNodes

In [3]:
depth(id::Int64) = floor(Int64, log2(id))
depth(n::MyNodes.BTNode) = depth(n.id)
children_ids(id::Int64) = [2*id, 2*id+1]
AbstractTrees.children(n::MyNodes.BTNode) = expand(n) ? MyNodes.BTNode.(children_ids(n.id), n.expand_depth) : MyNodes.BTNode[]
expand(n::MyNodes.BTNode) = depth(n) < n.expand_depth
Base.show(io::IO, n::MyNodes.BTNode) = print(io, "$(n.id): d=$(depth(n)), $(children_ids(n.id))")

In [4]:
for i in 1:18
    n = MyNodes.BTNode(i)
    println(n, " Expand: ", expand(n))
end

1: d=0, [2, 3] Expand: true
2: d=1, [4, 5] Expand: true
3: d=1, [6, 7] Expand: true
4: d=2, [8, 9] Expand: true
5: d=2, [10, 11] Expand: true
6: d=2, [12, 13] Expand: true
7: d=2, [14, 15] Expand: true
8: d=3, [16, 17] Expand: true
9: d=3, [18, 19] Expand: true
10: d=3, [20, 21] Expand: true
11: d=3, [22, 23] Expand: true
12: d=3, [24, 25] Expand: true
13: d=3, [26, 27] Expand: true
14: d=3, [28, 29] Expand: true
15: d=3, [30, 31] Expand: true
16: d=4, [32, 33] Expand: true
17: d=4, [34, 35] Expand: true
18: d=4, [36, 37] Expand: true


In [5]:
root = MyNodes.BTNode(1)

1: d=0, [2, 3]

In [6]:
print_tree(root)

1: d=0, [2, 3]
├─ 2: d=1, [4, 5]
│  ├─ 4: d=2, [8, 9]
│  └─ 5: d=2, [10, 11]
└─ 3: d=1, [6, 7]
   ├─ 6: d=2, [12, 13]
   └─ 7: d=2, [14, 15]


In [7]:
print(json(D3Tree(root)))
D3Tree(root)

{"children":[[2,5],[3,4],[],[],[6,7],[],[]],"text":["1: d=0, [2, 3]","2: d=1, [4, 5]","4: d=2, [8, 9]","5: d=2, [10, 11]","3: d=1, [6, 7]","6: d=2, [12, 13]","7: d=2, [14, 15]"],"tooltip":["1: d=0, [2, 3]","2: d=1, [4, 5]","4: d=2, [8, 9]","5: d=2, [10, 11]","3: d=1, [6, 7]","6: d=2, [12, 13]","7: d=2, [14, 15]"],"style":["","","","","","",""],"link_style":["","","","","","",""],"title":"Julia D3Tree","options":{}}

1: d=0, [2, 3]
├──2: d=1, [4, 5]
│  ├──4: d=2, [8, 9]
│  └──5: d=2, [10, 11]
└──3: d=1, [6, 7]
   ├──6: d=2, [12, 13]
   └──7: d=2, [14, 15]

In [9]:
leaf = MyNodes.BTNode(4) 
print(json(D3Tree(leaf)))
D3Tree(MyNodes.BTNode(4))

{"children":[[2,5],[3,4],[],[],[6,7],[],[]],"text":["4: d=2, [8, 9]","8: d=3, [16, 17]","16: d=4, [32, 33]","17: d=4, [34, 35]","9: d=3, [18, 19]","18: d=4, [36, 37]","19: d=4, [38, 39]"],"tooltip":["4: d=2, [8, 9]","8: d=3, [16, 17]","16: d=4, [32, 33]","17: d=4, [34, 35]","9: d=3, [18, 19]","18: d=4, [36, 37]","19: d=4, [38, 39]"],"style":["","","","","","",""],"link_style":["","","","","","",""],"title":"Julia D3Tree","options":{}}

4: d=2, [8, 9]
├──8: d=3, [16, 17]
│  ├──16: d=4, [32, 33]
│  └──17: d=4, [34, 35]
└──9: d=3, [18, 19]
   ├──18: d=4, [36, 37]
   └──19: d=4, [38, 39]

# Sockets and JavaScript

Resources:

- js: https://developer.mozilla.org/en-US/docs/Web/JavaScript/A_re-introduction_to_JavaScript
- js for Python programmers: https://mike.depalatis.net/blog/javascript-for-python-programmers.html
- js websockets example: https://www.geeksforgeeks.org/html-content-modification-using-javascript/
- js websockets docs: https://developer.mozilla.org/en-US/docs/Web/API/WebSockets_API/Writing_WebSocket_client_applications
- Julia websockets: https://github.com/JuliaWeb/HTTP.jl/blob/master/src/WebSockets.jl

Dealing with websockets and json: 
 - using uri in websocket: https://stackoverflow.com/questions/62473938/whats-the-uri-path-component-in-websocket-used-for
 - Fucked up error messages in JSON:

In [2]:
using HTTP

In [2]:
responses = Dict("a"=>"1", "b"=>"2")
get(responses, "a", "error")

"1"

## Testing javascript

In [12]:
task = @async HTTP.WebSockets.open("ws://127.0.0.1:8081") do ws
#     write(ws, "b")
    x = readavailable(ws)
    @show x
    println(String(x))
end;

In [46]:
function Base.show(f::IO, m::MIME"text/html", n::MyNodes.BTNode)
    html_string = """
    <!-- client.html -->
    <!-- https://developer.mozilla.org/en-US/docs/Web/API/WebSocket#examples -->
    <html>
    <body>

    <script>

    function changeStuff() {
        document.getElementById("myDisplay").innerHTML="XXX";
    }    
    
    const socket = new WebSocket('ws://127.0.0.1:8081');
    
    // Connect
    socket.addEventListener('open', function (event) {
        socket.send('Hello Server!');
    });
    
    // Handle server messages
    socket.addEventListener('message', function (event) {
        alert(['Message from server ', event.data]);
    });
    
    function sendStuff() {
        changeStuff();
        socket.send("Here's some text that the server is urgently awaiting!");
    }

    </script>
    
    <p id="myDisplay">
    </p>
    
    
    <button type="button" onclick="sendStuff()">
        Run!
    </button>


    </body>
    </html>
    """
    println(f,html_string)
end

MyNodes.BTNode(2,3)

1
READY


2: d=1, [4, 5]

In [107]:
# get UID
root=MyNodes.BTNode(1)
string(objectid(root))

"16422008291103055432"

In [169]:
Vector{UInt8}(json(Dict("response"=>"ok")))

17-element Vector{UInt8}:
 0x7b
 0x22
 0x72
 0x65
 0x73
 0x70
 0x6f
 0x6e
 0x73
 0x65
 0x22
 0x3a
 0x22
 0x6f
 0x6b
 0x22
 0x7d

In [11]:
# Closing server will stop HTTP.listen as well as WebSockets.listen
# close(server)

server = Sockets.listen(Sockets.localhost, UInt16(8098))

# Internally, WebSockets.listen is just HTTP.listen. When passed server, the ip and port are ignored.
@async HTTP.WebSockets.listen("1.0.0.1", UInt16(1001); verbose=true, server = server) do ws
#     println("Starting...")
    println(HTTP.Messages.uri(ws.request))
#     println(typeof(ws.request))
#     println(ws.request)
#     println(ws.request.target) ### This is the path part of the URI
#     println("==============")
    i=0
    while !eof(ws) 
        data = readavailable(ws)  
#         data = read(ws, UInt8)
        i+=1
        println(i)
        read_data=String(data)
        if length(read_data)>0
            println("Msg (parsed): ", JSON.parse(read_data))
        else
            println("Zero length data: ", data)
            println(ws.request.target)
        end
#         println("Msg (raw): ", Vector{UInt8}(data), "\nMsg: ", String(data))
#         println("Msg (parsed): ", JSON.parse())
        
        response = json(Dict("response"=>"ok"))
        write(ws, response)
    end
    println("Done.")
end

┌ Info: Listening on: 127.0.0.1:8098
└ @ HTTP.Servers /home/mrkos/projects/D3Trees.jl/dev/HTTP/src/Servers.jl:268


Task (runnable) @0x00007fb819d7d5a0

In [16]:
tree_json = json(root)
codeunits(tree_json);

In [17]:
HTTP.WebSockets.open("ws://127.0.0.1:8098/peeees") do ws
    write(ws, tree_json)
    x = readavailable(ws)
    @show x
    println(String(x))
end;

/peeees
1
Msg (parsed): Dict{String, Any}("expand_depth" => 2, "id" => 1)
x = UInt8[0x7b, 0x22, 0x72, 0x65, 0x73, 0x70, 0x6f, 0x6e, 0x73, 0x65, 0x22, 0x3a, 0x22, 0x6f, 0x6b, 0x22, 0x7d]
{"response":"ok"}
2
Zero length data: UInt8[]
/peeees
Done.


## Testing websockets

In [329]:
close(server)

server = Sockets.listen(Sockets.localhost, UInt16(8098))
@async HTTP.WebSockets.listen("1.0.0.1", UInt16(1001); verbose=true, server = server) do ws
    i=0
    while !eof(ws) 
        i+=1
        println(i)
        data = readavailable(ws)  
        read_data=String(data)
        if length(read_data)>0
            println("Msg (parsed): ", JSON.parse(read_data))
        else
            println("Zero length data: ", data)
        end
        response = json(Dict("response"=>"ok"))
        write(ws, response)
    end
end

┌ Info: Listening on: 127.0.0.1:8098
└ @ HTTP.Servers /home/mrkos/projects/D3Trees.jl/dev/HTTP/src/Servers.jl:268


Task (runnable) @0x00007efc3eb9d5a0

In [332]:
HTTP.WebSockets.open("ws://127.0.0.1:8098") do ws
    write(ws, json(Dict("message"=>"Hello")))
    x = readavailable(ws)
    println(String(x))
end;

1
Zero length data: UInt8[]


In [287]:
HTTP.WebSockets.open("ws://127.0.0.1:8098") do ws
    write(ws, json(Dict("message"=>"Hi")))
    x = readavailable(ws)
    @show x
    println(String(x))
end;

1
Msg (parsed): Dict{String, Any}("message" => "Hi")
x = UInt8[0x7b, 0x22, 0x72, 0x65, 0x73, 0x70, 0x6f, 0x6e, 0x73, 0x65, 0x22, 0x3a, 0x22, 0x6f, 0x6b, 0x22, 0x7d]
{"response":"ok"}
2
Zero length data: UInt8[]


In [16]:
uri = HTTP.URI("http://localhost:8081/pes")
uri.path

"/pes"

In [153]:
tree_json = json(root)
codeunits(tree_json);

In [123]:
vals = JSON.parse(string(tree_json))
vals["id"]

1

In [160]:
JSON.parse("{\"id\":1,\"expand_depth\":2}\n")

Dict{String, Any} with 2 entries:
  "expand_depth" => 2
  "id"           => 1

In [16]:
""

""

In [ ]:
ex = InterruptException()
Base.throwto(server, ex)
# close(http.sock)

In [ ]:
@async print("aha")

In [20]:
HTTP.WebSockets.open("ws://127.0.0.1:8082") do ws
    write(ws, "Foo")
end

HTTP.Messages.Response:
"""
HTTP/1.1 101 Switching Protocols
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Accept: d/B9qloNmU3/rK/eOLmp9gThPEk=

"""

In [10]:
?HTTP.WebSockets

No docstring or readme file found for module `HTTP.WebSockets`.

Module does not export any names.


In [20]:
t = @task begin; sleep(1); println("done"); end

done


Task (runnable) @0x00007f92c21da6e0

In [12]:
t

Task (runnable) @0x00007f9293c755a0

In [19]:
schedule(t)

Task (runnable) @0x00007f92c21da290

In [220]:
Dict("a"=>1)

Dict{String, Int64} with 1 entry:
  "a" => 1

In [224]:
json(Dict("a"=>1))

"{\"a\":1}"

In [241]:
length([1,2])

2

In [56]:
#WebSocket Examples
port=8099
@async HTTP.WebSockets.listen("127.0.0.1", UInt16(port)) do ws
    while !eof(ws)
        data = readavailable(ws)
        if length(data)>0
            JSON.parse(String(data))
        end
        write(ws, json(Dict("ok"=>"yes")))
    end
end

HTTP.WebSockets.open("ws://127.0.0.1:$port") do ws
    write(ws, json(Dict("a"=>1, "b"=>"val")))
    x = readavailable(ws)
    @show x
    println(String(x))
end;

2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
1
1
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
2
1
READY
x = UInt8[0x7b, 0x22, 0x6f, 0x6b, 0x22, 0x3a, 0x22, 0x79, 0x65, 0x73, 0x22, 0x7d]
{"ok":"yes"}


┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#116#118")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[55]:9, upgrade(f::var"#116#118", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#116#118"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_tran

## POC: Pass json bidirectionally from julia to javascript
exchange simple, static json files and parse them on both sides

Reconnect websocket:

 - https://stackoverflow.com/questions/13797262/how-to-reconnect-to-websocket-after-close-connection
 - https://stackoverflow.com/questions/3780511/reconnection-of-client-when-server-reboots-in-websocket

In [9]:
PORT = 8081

8081

In [38]:
close(server)
c = Channel(100)

server = Sockets.listen(Sockets.localhost, UInt16(PORT))
@async HTTP.WebSockets.listen("0.0.0.0", UInt16(1001); verbose=true, server = server) do ws
    i=0
    while !eof(ws) 
        i+=1
        println(i)
        data = readavailable(ws)  
        read_data=String(data)
        println(read_data)
        put!(c, read_data)
#         printnln(read_data)
#         if data=="READY"
#             println("\tReady message parsed!")
#         elseif length(read_data)>0
#             println("Json received (parsed): ", JSON.parse(read_data))
#         else
#             println("Zero length data: ", data)
#         end
        response = json(Dict("response"=>"ok", "i"=>i))
        write(ws, response)
    end
end

┌ Info: Listening on: 127.0.0.1:8081
└ @ HTTP.Servers /home/mrkos/projects/D3Trees.jl/dev/HTTP/src/Servers.jl:268


Task (runnable) @0x00007f75b680b540

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#10#12")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[36]:10, upgrade(f::var"#10#12", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#10#12"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transacti

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#18#20")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[38]:10, upgrade(f::var"#18#20", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#18#20"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transacti

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#18#20")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[38]:10, upgrade(f::var"#18#20", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#18#20"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transacti

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#18#20")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[38]:10, upgrade(f::var"#18#20", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#18#20"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transacti

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#18#20")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[38]:10, upgrade(f::var"#18#20", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#18#20"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transacti

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#18#20")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[38]:10, upgrade(f::var"#18#20", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#18#20"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transacti

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#18#20")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[38]:10, upgrade(f::var"#18#20", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#18#20"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transacti

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#18#20")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[38]:10, upgrade(f::var"#18#20", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#18#20"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transacti

In [31]:
take!(c)

3
{"name":"aha","depth":3,"j":2}


"{\"name\":\"aha\",\"depth\":3,\"j\":2}"

4
1
READY
2
1
READY
2
1
READY
2


┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#6#8")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[28]:10, upgrade(f::var"#6#8", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#6#8"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transaction,), 

In [61]:
HTTP.WebSockets.open("ws://127.0.0.1:$PORT") do ws
    write(ws, json(Dict("a"=>1, "b"=>"val")))
    x = readavailable(ws)
    @show x
    println(String(x))
end;

1
{"b":"val","a":1}
x = UInt8[0x7b, 0x22, 0x72, 0x65, 0x73, 0x70, 0x6f, 0x6e, 0x73, 0x65, 0x22, 0x3a, 0x22, 0x6f, 0x6b, 0x22, 0x2c, 0x22, 0x69, 0x22, 0x3a, 0x31, 0x7d]
{"response":"ok","i":1}
2



In [47]:
function Base.show(f::IO, m::MIME"text/html", n::MyNodes.BTNode)
    html_string = """
    <html>
    <body>

    <script>

    function setDisplay(message) {
        document.getElementById("myDisplay").innerHTML=message;
    }    
    
    var socket = new WebSocket('ws://127.0.0.1:$PORT');
    
    // Connect
    socket.addEventListener('open', function (event) {
        socket.send('READY');
        setDisplay("Open");
    });
    
    // Handle server messages
    socket.addEventListener('message', function (event) {
        alert(['Message from server ', event.data]);
        setDisplay("Data received.");
    });
    
    function sendStuff() {
        let obj = new Object();
        obj.name="aha";
        obj.depth=3;
        socket.send(JSON.stringify(obj));
    
    }
    
    function closeSocket() {
        document.getElementById("myDisplay").innerHTML="Close?";
    }
    </script>
        
    <button type="button" onclick="sendStuff()">
        Send!
    </button>
    
    <button type="button" onclick="closeSocket()">
        Close socket.
    </button>
    
    <p id="myDisplay">
        Initializing...
    </p>


    </body>
    </html>
    """
    println(f,html_string)
end

MyNodes.BTNode(2,3)

2: d=1, [4, 5]

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#84#86")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[42]:9, upgrade(f::var"#84#86", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#84#86"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transactio

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#84#86")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[42]:9, upgrade(f::var"#84#86", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#84#86"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transactio

┌ Error: error handling request
│   exception = (HTTP.WebSockets.WebSocketError(0x03e9, "Status: Going Away, Internal Code: "), Base.StackTraces.StackFrame[_readframe(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:311, readmessage(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at WebSockets.jl:260, readavailable at WebSockets.jl:257 [inlined], (::var"#84#86")(ws::HTTP.WebSockets.WebSocket{HTTP.ConnectionPool.Connection}) at In[42]:9, upgrade(f::var"#84#86", http::HTTP.Streams.Stream{HTTP.Messages.Request{Vector{UInt8}}, HTTP.ConnectionPool.Connection}; binary::Bool) at WebSockets.jl:160, upgrade at WebSockets.jl:142 [inlined], #6 at WebSockets.jl:136 [inlined], handle_transaction(f::HTTP.WebSockets.var"#6#7"{Bool, var"#84#86"}, c::HTTP.ConnectionPool.Connection, server::HTTP.Servers.Server{Nothing, Sockets.TCPServer}; final_transaction::Bool) at Servers.jl:416, (::HTTP.Servers.var"#handle_transaction##kw")(::NamedTuple{(:final_transactio

## Plan for real implementation:
Protocol: 
 1. J: start server
 2. js: connect to socket, send ready message when ready
 3. J: send initial tree structure in json
 4. js: when not-yet loaded node is expanded, send request to server
 5. J: respond by generating the subtree and send it to the visualization